In [1]:
import pandas as pd
import numpy as np

In [3]:
org_df=pd.read_csv('adm_organizations.csv').drop_duplicates(['TID', 'organization_name'], keep='first')
exclude=['Пункт выдачи', 'Достопримечательность', 'Мемориальная доска, закладной камень', 'Спортплощадка',
        'Памятник технике', 'Жанровая скульптура', 'Банк, точка банковского обслуживания', 'Wi-Fi хот-спот',
        'Туалет для инвалидов', 'Туалет', 'Гражданская оборона', 'НИИ', 'Городская телефонная сеть', 
        'Кофейный автомат', 'Станция зарядки электромобилей', 'Остановка общественного транспорта', 
        'Пожарные части и службы', 'Банк, обмен валюты', 'Обмен валюты', 'Вендомат', 'Совет депутатов', 
        'Комната матери и ребенка', 'Пункт предрейсового осмотра', 'Прокуратура', 'Следственный комитет', 
        'Военкомат', 'Банк, денежные переводы', 'Отделение милиции', 'Постамат', 'Аренда зарядных устройств',
        'Банк', 'Отделение полиции', 'Общественная организация', 'Администрация', 'Продажа воды', 
        'Финансовые услуги на кассе', 'Жилой комплекс', 'Пекарня', 'Услуги частных специалистов',
        'Офис организации', 'Водомат', 'Министерства, ведомства, государственные службы',
        'Общественный пункт охраны порядка']
org_df=org_df[org_df.description.notnull()]
org_df=org_df[~org_df.description.isin(exclude)]

In [5]:
count_org_df=org_df.copy()
count_org_df['num_org_found']=1
count_org_df.groupby('TID').num_org_found.sum().to_frame().to_csv('num_org_found.csv')

In [7]:
count_org_df=org_df.copy()
count_org_df['num_org_found']=1
count_org_df=count_org_df.groupby('description').num_org_found.sum().to_frame().sort_values(
                            by='num_org_found', ascending=False)
count_org_df=count_org_df[count_org_df.index.str.lower().str.contains('магазин')]
count_org_df.to_excel('num_org_by_category_shop.xlsx')

In [9]:
cat_dict=pd.read_csv('categories.csv').set_index('description').category.to_dict()
count_org_df=org_df.copy()
#.merge(pd.read_csv('target_september_selected.csv'), on='TID', how='left')
count_org_df['num_org_found']=1
count_org_df['description']=count_org_df.description.apply(lambda d: cat_dict[d] if d in cat_dict else d)
#median_incomes=count_org_df.groupby('description').target_sept.median().rename('median_income')
count_org_df=count_org_df.drop_duplicates(['TID', 'description'], keep='first')
unique_orgs=count_org_df.groupby(['TID', 'description']).num_org_found.sum().to_frame(
    ).reset_index().drop_duplicates('TID').groupby('description').num_org_found.sum(
    ).rename('num_org_found_uniq')
pivot=count_org_df.groupby('description').num_org_found.sum().to_frame().sort_values(
                            by='num_org_found', ascending=False).join(unique_orgs)
incomes=pd.read_csv('target_september_selected.csv').set_index('TID')
top_descriptions=pivot.index[:60].tolist()
for d in top_descriptions:
    incomes=incomes.join(pd.Series(True, index=count_org_df[count_org_df.description==d].set_index('TID').index
                          ).rename(d))
    incomes[d]=incomes[d].fillna(False)

base=incomes.target_sept.std()
n_shuffles=100
variance_influence=[]
median=[]
p_value=[]
for d in top_descriptions:
    mask=np.array(incomes[d]==1)
    std_base=incomes.target_sept[mask].std()
    variance_influence.append(base/std_base)
    median.append(incomes.target_sept[mask].median())
    succ=0
    for i in range(n_shuffles):
        np.random.shuffle(mask)
        #print(mask)
        cur_std=incomes.target_sept[mask].std()
        if cur_std<=std_base:
            succ+=1
    p_value.append(succ/n_shuffles)
pivot.join(pd.Series(variance_influence, index=top_descriptions, name='variance_influence')).join(
        pd.Series(p_value, index=top_descriptions, name='p_value')).join(
        pd.Series(median, index=top_descriptions, name='median')).to_excel('num_org_by_category.xlsx')

/tmp/ipykernel_1350564/28581795.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  incomes[d]=incomes[d].fillna(False)
/tmp/ipykernel_1350564/28581795.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  incomes[d]=incomes[d].fillna(False)
/tmp/ipykernel_1350564/28581795.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  incomes[d]=in

In [11]:
short_list=["Бытовые услуги", "Общепит", "Магазин бытовой прочий", "Магазин бытовой продуктовый",
            "Салон красоты/барбершоп", "Центр развития/спортшкола", "Магазин цветов", "Магазин строительный",
            "Магазин парфюмерия/бытовая химия", "Алкогольные напитки", "Автоуслуги", 
            "Дополнительное образование/курсы/мастреклассы", "Магазин электроники/бытовой техники",
            "Недвижимость", "Магазин оптики", "Ювелирно-ломбардная категория", "Магазин мебели",
            "Коммунальная служба", "Туруслуги", "Зооуслуги", "Химчистка", "Спорт/фитнес", "Зоомагазин"]
for d in short_list:
    count_org_df[d]=(count_org_df.description==d).astype(int)
gb=count_org_df.groupby('TID')
pd.concat([(gb[d].sum()>0).astype(int) for d in short_list], axis=1).to_csv('org_features.csv')